In [1]:
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from imutils import paths
import random
import cv2
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import flwr as fl
# Load and compile Keras model


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)


In [3]:
BATCH_SIZE = 8
train_data_dir = 'D:/TQ/Federated/1/train/'

test_data_dir = 'D:/TQ/Federated/1/val/'


In [4]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Set the validation split percentage
)
test_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)




In [5]:
print("Class Counts in Training Samples:")
total_classes = sorted(os.listdir(train_data_dir))
print(total_classes)
for category in total_classes:
    category_path = os.path.join(train_data_dir, category)
    sample_count = len(os.listdir(category_path))
    print(f"{category}: {sample_count} samples")


Class Counts in Training Samples:
['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
Animal_Abuse: 124 samples
Arson: 80 samples
Fight: 320 samples
Normal: 199 samples
Riot: 251 samples
Traffic_Accident: 160 samples
Train_Accident: 440 samples


In [6]:
train_classes = sorted(os.listdir(train_data_dir))
test_classes = sorted(os.listdir(test_data_dir))


In [7]:
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Freeze the base model layers
base_model.trainable = False

# Add custom classification layers on top of the base model
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Add dropout layer with a dropout rate of 0.5
outputs = Dense(7, activation='softmax')(x)  # Assuming binary classification

# Create the full model
model1 = Model(inputs, outputs)
optimizer = Adam(learning_rate=0.001)
model1.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\random.py:370: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return self.randrange(a, b+1)


In [8]:
traingen = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(225, 225),
                                                   class_mode='categorical',
                                                   classes=total_classes,
                                                   subset='training',
                                                   batch_size=BATCH_SIZE, 
                                                   shuffle=True,
                                                   seed=42)

validgen = train_datagen.flow_from_directory(train_data_dir,
                                               target_size=(225, 225),
                                               class_mode='categorical',
                                               classes=total_classes,
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

testgen = test_datagen.flow_from_directory(test_data_dir,
                                             target_size=(225, 225),
                                             class_mode='categorical',
                                             classes=total_classes,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)


Found 1261 images belonging to 7 classes.
Found 313 images belonging to 7 classes.
Found 360 images belonging to 7 classes.


In [9]:
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
callbacks = [early_stopping]


In [10]:
history = model1.fit(
            traingen,
            epochs=5,
            steps_per_epoch=len(traingen),
            validation_data=validgen,
            validation_steps=len(validgen),
            callbacks=callbacks,
            verbose = 1
)

C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:2616: DeprecationWarning: Please import `affine_transform` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  ndimage.interpolation.affine_transform(


Epoch 1/5
158/158 [==============================] - 52s 305ms/step - loss: 0.8215 - accuracy: 0.7193 - val_loss: 1.3026 - val_accuracy: 0.6102
Epoch 2/5
158/158 [==============================] - 29s 183ms/step - loss: 0.2309 - accuracy: 0.9318 - val_loss: 1.6398 - val_accuracy: 0.6454
Epoch 3/5
158/158 [==============================] - 28s 178ms/step - loss: 0.1392 - accuracy: 0.9564 - val_loss: 1.8614 - val_accuracy: 0.6358
Epoch 4/5
158/158 [==============================] - 27s 170ms/step - loss: 0.1032 - accuracy: 0.9730 - val_loss: 2.0864 - val_accuracy: 0.6422
Epoch 5/5
158/158 [==============================] - 27s 172ms/step - loss: 0.0882 - accuracy: 0.9794 - val_loss: 1.9256 - val_accuracy: 0.6677


In [11]:
#model = tf.keras.models.load_model("Client1_Centralized.h5")

In [12]:
y_true = testgen.classes

# Calculate predictions on the test data
y_pred = model1.predict(testgen)
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the predicted class labels

360/360 [==============================] - 9s 25ms/step


In [50]:
print("Accuracy:")
test_loss, test_accuracy = model1.evaluate(testgen, steps=len(testgen), verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Accuracy:


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:2616: DeprecationWarning: Please import `affine_transform` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  ndimage.interpolation.affine_transform(


Test Loss: 0.3338848650455475
Test Accuracy: 0.9388889074325562


In [51]:
# Calculate the confusion matrix
cm2 = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix Before Aggregation:")
print(cm2)

Confusion Matrix Before Aggregation:
[[28  0  0  3  0  0  0]
 [ 0 20  0  0  0  0  0]
 [ 0  0 67 13  0  0  0]
 [ 0  0  0 45  0  5  0]
 [ 0  0  0  0 63  0  0]
 [ 0  0  0  0  0 41  0]
 [ 0  0  0 21  0  6 48]]


In [52]:
import pandas as pd
from openpyxl import Workbook

categories = ['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
df_cm = pd.DataFrame(cm2, index=categories, columns=categories)

# Write the confusion matrix DataFrame to an Excel file
excel_writer = pd.ExcelWriter('D:/TQ/Federated/confusion_matrix.xlsx')
# Create an Excel workbook and write the confusion matrix DataFrame to a sheet
wb = Workbook()
ws = wb.active
ws.title = 'Confusion Matrix'

# Write the column names
ws.append([''] + df_cm.columns.tolist())

# Write the confusion matrix data
for index, row in df_cm.iterrows():
    ws.append([index] + row.tolist())

# Save the workbook to an Excel file
wb.save('D:/TQ/Federated/confusion_matrix1.xlsx')

print("Confusion matrix written to confusion_matrix.xlsx")

Confusion matrix written to confusion_matrix.xlsx


In [53]:
from sklearn.metrics import classification_report 
print(classification_report(y_true, y_pred_labels, target_names=total_classes))


                  precision    recall  f1-score   support

    Animal_Abuse       1.00      0.90      0.95        31
           Arson       1.00      1.00      1.00        20
           Fight       1.00      0.84      0.91        80
          Normal       0.55      0.90      0.68        50
            Riot       1.00      1.00      1.00        63
Traffic_Accident       0.79      1.00      0.88        41
  Train_Accident       1.00      0.64      0.78        75

        accuracy                           0.87       360
       macro avg       0.91      0.90      0.89       360
    weighted avg       0.91      0.87      0.87       360



In [54]:
from sklearn.metrics import confusion_matrix
import numpy as np


# We will store the results in a dictionary for easy access later
per_class_accuracies = {}

# Calculate the accuracy for each one of our classes
for idx, cls in enumerate(total_classes):
    true_positives = cm2[idx, idx]
    
    # Total instances of the current class
    total_instances = np.sum(cm[idx, :])
    
    # The accuracy for the current class is the ratio between correct predictions to all predictions
    per_class_accuracies[cls] = true_positives / total_instances if total_instances > 0 else 0
    print(f"{cls} - True Positives: {true_positives}, Total Instances: {total_instances}")

print(per_class_accuracies)

Animal_Abuse - True Positives: 28, Total Instances: 31
Arson - True Positives: 20, Total Instances: 20
Fight - True Positives: 67, Total Instances: 80
Normal - True Positives: 45, Total Instances: 50
Riot - True Positives: 63, Total Instances: 63
Traffic_Accident - True Positives: 41, Total Instances: 41
Train_Accident - True Positives: 48, Total Instances: 75
{'Animal_Abuse': 0.9032258064516129, 'Arson': 1.0, 'Fight': 0.8375, 'Normal': 0.9, 'Riot': 1.0, 'Traffic_Accident': 1.0, 'Train_Accident': 0.64}


In [18]:
model1.save("Client1_Centralized.h5")

In [38]:
client1 = tf.keras.models.load_model('D:/TQ/Federated/client1_30_fedavg.h5')

C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\random.py:370: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return self.randrange(a, b+1)


In [39]:
y_true = testgen.classes

# Calculate predictions on the test data
y_pred = client1.predict(testgen)
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the predicted class labels
test_loss, test_accuracy = client1.evaluate(testgen, steps=len(testgen), verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

print(classification_report(y_true, y_pred_labels, target_names=total_classes))


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:2616: DeprecationWarning: Please import `affine_transform` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  ndimage.interpolation.affine_transform(


360/360 [==============================] - 8s 20ms/step
Test Loss: 0.5148042440414429
Test Accuracy: 0.8916666507720947
                  precision    recall  f1-score   support

    Animal_Abuse       1.00      0.90      0.95        31
           Arson       1.00      1.00      1.00        20
           Fight       1.00      0.84      0.91        80
          Normal       0.55      0.90      0.68        50
            Riot       1.00      1.00      1.00        63
Traffic_Accident       0.79      1.00      0.88        41
  Train_Accident       1.00      0.64      0.78        75

        accuracy                           0.87       360
       macro avg       0.91      0.90      0.89       360
    weighted avg       0.91      0.87      0.87       360



In [40]:
cm2 = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix After Aggregation:")
print(cm2)

Confusion Matrix After Aggregation:
[[28  0  0  3  0  0  0]
 [ 0 20  0  0  0  0  0]
 [ 0  0 67 13  0  0  0]
 [ 0  0  0 45  0  5  0]
 [ 0  0  0  0 63  0  0]
 [ 0  0  0  0  0 41  0]
 [ 0  0  0 21  0  6 48]]


In [41]:
import pandas as pd
from openpyxl import Workbook

categories = ['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
df_cm = pd.DataFrame(cm, index=categories, columns=categories)

# Write the confusion matrix DataFrame to an Excel file
excel_writer = pd.ExcelWriter('D:/TQ/Federated/confusion_matrix.xlsx')
# Create an Excel workbook and write the confusion matrix DataFrame to a sheet
wb = Workbook()
ws = wb.active
ws.title = 'Confusion Matrix'

# Write the column names
ws.append([''] + df_cm.columns.tolist())

# Write the confusion matrix data
for index, row in df_cm.iterrows():
    ws.append([index] + row.tolist())

# Save the workbook to an Excel file
wb.save('D:/TQ/Federated/confusion_matrix1f.xlsx')

print("Confusion matrix written to confusion_matrix.xlsx")

Confusion matrix written to confusion_matrix.xlsx


In [45]:
from sklearn.metrics import confusion_matrix
import numpy as np


# We will store the results in a dictionary for easy access later
per_class_accuracies = {}

# Calculate the accuracy for each one of our classes
for idx, cls in enumerate(total_classes):
    true_positives = cm[idx, idx]
    
    # Total instances of the current class
    total_instances = np.sum(cm[idx, :])
    
    # The accuracy for the current class is the ratio between correct predictions to all predictions
    per_class_accuracies[cls] = true_positives / total_instances if total_instances > 0 else 0
    print(f"{cls} - True Positives: {true_positives}, Total Instances: {total_instances}")

print(per_class_accuracies)

Animal_Abuse - True Positives: 30, Total Instances: 31
Arson - True Positives: 20, Total Instances: 20
Fight - True Positives: 68, Total Instances: 80
Normal - True Positives: 44, Total Instances: 50
Riot - True Positives: 63, Total Instances: 63
Traffic_Accident - True Positives: 41, Total Instances: 41
Train_Accident - True Positives: 52, Total Instances: 75
{'Animal_Abuse': 0.967741935483871, 'Arson': 1.0, 'Fight': 0.85, 'Normal': 0.88, 'Riot': 1.0, 'Traffic_Accident': 1.0, 'Train_Accident': 0.6933333333333334}


In [43]:
print(per_class_accuracies)

{'Animal_Abuse': 0.9916666666666667, 'Arson': 1.0, 'Fight': 0.9638888888888889, 'Normal': 0.8944444444444445, 'Riot': 1.0, 'Traffic_Accident': 0.975, 'Train_Accident': 0.925}
